<a href="https://colab.research.google.com/github/Kush-Singh-26/NLP/blob/main/ChargenerateRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O input.txt

with open('input.txt', 'r') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)


--2025-05-03 19:41:34--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-05-03 19:41:34 (24.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
char2idx = {ch: idx for idx, ch in enumerate(chars)}
idx2char = {idx: ch for ch, idx in char2idx.items()}
encoded_text = torch.tensor([char2idx[c] for c in text], dtype=torch.long)

In [13]:
block_size = 100
batch_size = 64
hidden_size = 256
num_layers = 2
learning_rate = 0.002
num_epochs = 500

In [5]:
def get_batch(encoded_text, block_size, batch_size):
    ix = torch.randint(0, len(encoded_text) - block_size - 1, (batch_size,))
    x = torch.stack([encoded_text[i:i+block_size] for i in ix])
    y = torch.stack([encoded_text[i+1:i+block_size+1] for i in ix])
    return x, y

In [6]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden


In [14]:
model = CharRNN(vocab_size, hidden_size, num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

print("Training started...")
for epoch in range(num_epochs):
    x_batch, y_batch = get_batch(encoded_text, block_size, batch_size)
    model.train()
    optimizer.zero_grad()
    logits, _ = model(x_batch)
    loss = loss_fn(logits.view(-1, vocab_size), y_batch.view(-1))
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")


Training started...
Epoch 2/500, Loss: 3.7997
Epoch 4/500, Loss: 3.1649
Epoch 6/500, Loss: 2.9249
Epoch 8/500, Loss: 2.7971
Epoch 10/500, Loss: 2.6515
Epoch 12/500, Loss: 2.5842
Epoch 14/500, Loss: 2.5096
Epoch 16/500, Loss: 2.4375
Epoch 18/500, Loss: 2.3958
Epoch 20/500, Loss: 2.3650
Epoch 22/500, Loss: 2.3448
Epoch 24/500, Loss: 2.3034
Epoch 26/500, Loss: 2.2964
Epoch 28/500, Loss: 2.2738
Epoch 30/500, Loss: 2.2388
Epoch 32/500, Loss: 2.2241
Epoch 34/500, Loss: 2.1808
Epoch 36/500, Loss: 2.1766
Epoch 38/500, Loss: 2.1217
Epoch 40/500, Loss: 2.1111
Epoch 42/500, Loss: 2.0957
Epoch 44/500, Loss: 2.1444
Epoch 46/500, Loss: 2.1087
Epoch 48/500, Loss: 2.0835
Epoch 50/500, Loss: 2.0802
Epoch 52/500, Loss: 2.0588
Epoch 54/500, Loss: 2.0814
Epoch 56/500, Loss: 2.0406
Epoch 58/500, Loss: 1.9893
Epoch 60/500, Loss: 2.0542
Epoch 62/500, Loss: 2.0094
Epoch 64/500, Loss: 2.0056
Epoch 66/500, Loss: 1.9934
Epoch 68/500, Loss: 1.9296
Epoch 70/500, Loss: 1.9320
Epoch 72/500, Loss: 1.9687
Epoch 74/500

In [15]:
def generate(model, start_text='Once upon a ', length=300):
    model.eval()
    chars = list(start_text)
    input_seq = torch.tensor([char2idx[c] for c in chars], dtype=torch.long).unsqueeze(0)
    hidden = None

    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        last_logits = output[0, -1, :]
        probs = torch.softmax(last_logits, dim=0)
        next_idx = torch.multinomial(probs, num_samples=1).item()
        chars.append(idx2char[next_idx])
        input_seq = torch.tensor([[next_idx]], dtype=torch.long)

    return ''.join(chars)

In [16]:
print("\nGenerated text:")
print(generate(model, start_text="Once upon a time,"))


Generated text:
Once upon a time,
So sleall but my Larcius,
Seen for heaven?
O ginders
With rupe of Richurching of Warwipps and all this dauping I revight.

Born, farther:
Upon the wordst me, thensel my come hundrognautions mays and the unforce.
Seelf anwill'tes, but prokes, sweet sir; here presence
With my adveritions, my bots. I 


In [18]:
torch.save({
    'model_state_dict': model.state_dict(),
    'char2idx': char2idx,
    'idx2char': idx2char,
    'vocab_size': vocab_size,
    'hidden_size': hidden_size,
    'num_layers': num_layers
}, 'char_rnn_full.pt')


```python
checkpoint = torch.load('char_rnn_full.pt')

# Recreate model using saved hyperparameters
model = CharRNN(
    vocab_size=checkpoint['vocab_size'],
    hidden_size=checkpoint['hidden_size'],
    num_layers=checkpoint['num_layers']
)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

char2idx = checkpoint['char2idx']
idx2char = checkpoint['idx2char']

```